In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
import dtuimldmtools

OSError: [WinError 126] Det angivne modul blev ikke fundet. Error loading "c:\Users\malth\Miniconda3\envs\02450_ML\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.